# Ice Hockey Player Tracking Pipeline - Google Colab

Complete setup with all model downloads and verification.

## Models Summary

| Model | Size | Source | Status |
|-------|------|--------|--------|
| `hockey_yolo.pt` | ~140 MB | **Your custom** | Manual upload |
| `sam2.1_hiera_large.pt` | ~898 MB | Meta AI | Auto-download |
| `cutie-base-mega.pth` | ~507 MB | GitHub | Auto-download |
| `vitpose-h.pth` | ~1.1 GB | OpenMMLab | Auto-download |
| `parseq_hockey.ckpt` | ~100 MB | **Your custom** | Manual upload |
| `legibility_resnet34_hockey.pth` | ~85 MB | **Your custom** | Manual upload |
| SigLIP | ~400 MB | HuggingFace | Auto-download |

---
## Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

---
## Step 2: Set Home Directory

In [ ]:
import os
HOME = "/content"
os.chdir(HOME)
print(f"Working directory: {HOME}")

---
## Step 3: Clone Your Codebase from GitHub

**EDIT THE URL BELOW** to point to your GitHub repository:

In [ ]:
# ============================================
# EDIT THIS: Your GitHub repository URL
# ============================================
GITHUB_REPO = "https://github.com/YOUR_USERNAME/YOUR_REPO.git"

# Clone the repository
!git clone {GITHUB_REPO} {HOME}/cv

print(f"\nRepository cloned to {HOME}/cv")

In [ ]:
# Verify the clone
!ls -la {HOME}/cv/unified_pipeline/

---
## Step 4: Install Core Dependencies

In [ ]:
# Core packages
!pip install -q ultralytics opencv-python-headless pillow tqdm pyyaml omegaconf hydra-core
!pip install -q einops timm scipy scikit-learn pandas matplotlib
!pip install -q lap cython_bbox filterpy hdbscan
!pip install -q transformers accelerate  # For SigLIP
!pip install -q umap-learn  # For dimensionality reduction
!pip install -q supervision  # For visualization utilities

print("\nCore dependencies installed!")

---
## Step 5: Install & Download SAM2 Real-Time Fork

SAM2 for high-quality mask generation.

In [ ]:
# Clone SAM2 real-time fork
%cd {HOME}
!git clone https://github.com/Gy920/segment-anything-2-real-time.git

# Install SAM2
%cd {HOME}/segment-anything-2-real-time
!pip install -e . -q
!python setup.py build_ext --inplace

print("\nSAM2 real-time installed!")

In [ ]:
# Download SAM2 checkpoint (~898 MB)
!mkdir -p {HOME}/segment-anything-2-real-time/checkpoints

print("Downloading SAM2 checkpoint (898 MB)...")
!wget -q --show-progress -O {HOME}/segment-anything-2-real-time/checkpoints/sam2.1_hiera_large.pt \
    https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt

# Verify
!ls -lh {HOME}/segment-anything-2-real-time/checkpoints/sam2.1_hiera_large.pt

In [ ]:
# Test SAM2 loading
%cd {HOME}/segment-anything-2-real-time

from sam2.build_sam import build_sam2_camera_predictor

SAM2_CHECKPOINT = f"{HOME}/segment-anything-2-real-time/checkpoints/sam2.1_hiera_large.pt"
SAM2_CONFIG = f"{HOME}/segment-anything-2-real-time/configs/sam2.1/sam2.1_hiera_l.yaml"

print("Loading SAM2 predictor...")
predictor = build_sam2_camera_predictor(SAM2_CONFIG, SAM2_CHECKPOINT)
print("SAM2 loaded successfully!")

# Clean up to save memory
del predictor
torch.cuda.empty_cache()

---
## Step 6: Install & Download CUTIE

CUTIE for temporal mask propagation (handles occlusions).

In [ ]:
# Clone CUTIE
%cd {HOME}
!git clone https://github.com/hkchengrex/Cutie.git

# Install CUTIE
%cd {HOME}/Cutie
!pip install -e . -q

print("\nCUTIE installed!")

In [ ]:
# Download CUTIE weights (~507 MB)
!mkdir -p {HOME}/Cutie/weights

print("Downloading CUTIE weights (507 MB)...")
!wget -q --show-progress -O {HOME}/Cutie/weights/cutie-base-mega.pth \
    https://github.com/hkchengrex/Cutie/releases/download/v1.0/cutie-base-mega.pth

# Verify
!ls -lh {HOME}/Cutie/weights/cutie-base-mega.pth

---
## Step 7: Download ViTPose

ViTPose for pose estimation (used in jersey number recognition).

In [ ]:
# Install mmpose dependencies
!pip install -U openmim -q
!mim install mmengine -q
!mim install "mmcv>=2.0.0" -q
!mim install "mmdet>=3.0.0" -q
!mim install "mmpose>=1.0.0" -q

print("\nmmpose dependencies installed!")

In [ ]:
# Download ViTPose-H checkpoint (~1.1 GB)
!mkdir -p {HOME}/cv/unified_pipeline/models/vitpose

print("Downloading ViTPose-H (1.1 GB)...")
!wget -q --show-progress -O {HOME}/cv/unified_pipeline/models/vitpose/vitpose-h.pth \
    https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-huge_8xb64-210e_coco-256x192-e32adcd4_20230314.pth

# Verify
!ls -lh {HOME}/cv/unified_pipeline/models/vitpose/vitpose-h.pth

---
## Step 8: Download & Test SigLIP

SigLIP for team classification (auto-downloads from HuggingFace).

In [ ]:
# Optional: Set HuggingFace token if you have one
# Uncomment and add your token if needed

# import os
# os.environ['HF_TOKEN'] = 'your_huggingface_token_here'

# Or login interactively:
# from huggingface_hub import login
# login()

In [ ]:
# Download and test SigLIP (~400 MB)
print("Downloading SigLIP from HuggingFace (400 MB)...")

from transformers import AutoProcessor, SiglipVisionModel

SIGLIP_MODEL_PATH = 'google/siglip-base-patch16-224'

# This downloads the model on first run
siglip_model = SiglipVisionModel.from_pretrained(SIGLIP_MODEL_PATH)
siglip_processor = AutoProcessor.from_pretrained(SIGLIP_MODEL_PATH)

print("\nSigLIP loaded successfully!")
print(f"Model: {SIGLIP_MODEL_PATH}")

# Clean up to save memory
del siglip_model, siglip_processor
torch.cuda.empty_cache()

---
## Step 9: Create Model Directories for Your Custom Models

In [ ]:
# Create directories for your custom models
!mkdir -p {HOME}/cv/unified_pipeline/models
!mkdir -p {HOME}/cv/unified_pipeline/models/parseq
!mkdir -p {HOME}/cv/unified_pipeline/models/legibility

print("Model directories created:")
print(f"  {HOME}/cv/unified_pipeline/models/")
print(f"  {HOME}/cv/unified_pipeline/models/parseq/")
print(f"  {HOME}/cv/unified_pipeline/models/legibility/")
print("\n" + "="*60)
print("NOW UPLOAD YOUR CUSTOM MODELS (see next cells)")
print("="*60)

---
## Step 10: UPLOAD YOUR CUSTOM MODELS

### You need to upload these 3 models:

| Model | Upload To |
|-------|----------|
| `hockey_yolo.pt` | `/content/cv/unified_pipeline/models/hockey_yolo.pt` |
| `parseq_hockey.ckpt` | `/content/cv/unified_pipeline/models/parseq/parseq_hockey.ckpt` |
| `legibility_resnet34_hockey.pth` | `/content/cv/unified_pipeline/models/legibility/legibility_resnet34_hockey.pth` |

Run each cell below to upload each model:

In [ ]:
from google.colab import files
import shutil

print("="*60)
print("UPLOAD 1/3: hockey_yolo.pt")
print("="*60)
uploaded = files.upload()

for filename in uploaded.keys():
    dest = f'{HOME}/cv/unified_pipeline/models/hockey_yolo.pt'
    shutil.move(filename, dest)
    size = os.path.getsize(dest) / 1024**2
    print(f"\nSaved: {dest}")
    print(f"Size: {size:.1f} MB")

In [ ]:
print("="*60)
print("UPLOAD 2/3: parseq_hockey.ckpt")
print("="*60)
uploaded = files.upload()

for filename in uploaded.keys():
    dest = f'{HOME}/cv/unified_pipeline/models/parseq/parseq_hockey.ckpt'
    shutil.move(filename, dest)
    size = os.path.getsize(dest) / 1024**2
    print(f"\nSaved: {dest}")
    print(f"Size: {size:.1f} MB")

In [ ]:
print("="*60)
print("UPLOAD 3/3: legibility_resnet34_hockey.pth")
print("="*60)
uploaded = files.upload()

for filename in uploaded.keys():
    dest = f'{HOME}/cv/unified_pipeline/models/legibility/legibility_resnet34_hockey.pth'
    shutil.move(filename, dest)
    size = os.path.getsize(dest) / 1024**2
    print(f"\nSaved: {dest}")
    print(f"Size: {size:.1f} MB")

---
## Step 11: VERIFY ALL MODELS

**IMPORTANT:** Run this cell to verify all models are properly set up before proceeding!

In [ ]:
import os

models = {
    "YOLO Detection": f"{HOME}/cv/unified_pipeline/models/hockey_yolo.pt",
    "SAM2": f"{HOME}/segment-anything-2-real-time/checkpoints/sam2.1_hiera_large.pt",
    "SAM2 Config": f"{HOME}/segment-anything-2-real-time/configs/sam2.1/sam2.1_hiera_l.yaml",
    "CUTIE": f"{HOME}/Cutie/weights/cutie-base-mega.pth",
    "ViTPose": f"{HOME}/cv/unified_pipeline/models/vitpose/vitpose-h.pth",
    "PARSeq": f"{HOME}/cv/unified_pipeline/models/parseq/parseq_hockey.ckpt",
    "Legibility": f"{HOME}/cv/unified_pipeline/models/legibility/legibility_resnet34_hockey.pth",
}

print("="*70)
print("MODEL VERIFICATION")
print("="*70)

all_present = True
total_size = 0

for name, path in models.items():
    exists = os.path.exists(path)
    if exists:
        size = os.path.getsize(path) / 1024**2
        total_size += size
        status = f"OK ({size:.1f} MB)"
    else:
        status = "MISSING!"
        all_present = False
    print(f"{name:20s}: {status}")

print("="*70)
print(f"Total model size: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
print("="*70)

# Check SigLIP (cached in HuggingFace)
try:
    from transformers import SiglipVisionModel
    print("SigLIP:              OK (cached in HuggingFace)")
except:
    print("SigLIP:              MISSING!")
    all_present = False

print("="*70)
if all_present:
    print("\n ALL MODELS READY! You can proceed to run the pipeline.")
else:
    print("\n SOME MODELS ARE MISSING! Please upload/download them before proceeding.")

---
## Step 12: Upload Test Video

In [ ]:
print("Upload your test video (MP4):")
uploaded = files.upload()

VIDEO_PATH = None
for filename in uploaded.keys():
    VIDEO_PATH = f"{HOME}/{filename}"
    print(f"\nVideo uploaded: {VIDEO_PATH}")
    
    # Show video info
    import cv2
    cap = cv2.VideoCapture(VIDEO_PATH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    print(f"Resolution: {width}x{height}")
    print(f"FPS: {fps}")
    print(f"Frames: {frames}")
    print(f"Duration: {frames/fps:.1f}s")

---
## Step 13: Create Pipeline Configuration

In [ ]:
%cd {HOME}/cv

# Add to Python path
import sys
sys.path.insert(0, f'{HOME}/cv')
sys.path.insert(0, f'{HOME}/segment-anything-2-real-time')
sys.path.insert(0, f'{HOME}/Cutie')

# Create configuration YAML
config_yaml = f"""
detection:
  model_path: "{HOME}/cv/unified_pipeline/models/hockey_yolo.pt"
  imgsz: 1280
  player_confidence: 0.4
  puck_confidence: 0.2
  device: "cuda"

tracking:
  track_thresh: 0.6
  track_buffer: 30

mask:
  sam2_checkpoint: "{HOME}/segment-anything-2-real-time/checkpoints/sam2.1_hiera_large.pt"
  sam2_config: "{HOME}/segment-anything-2-real-time/configs/sam2.1/sam2.1_hiera_l.yaml"
  cutie_checkpoint: "{HOME}/Cutie/weights/cutie-base-mega.pth"
  max_internal_size: 540

jersey:
  str_model: "{HOME}/cv/unified_pipeline/models/parseq/parseq_hockey.ckpt"
  legibility_model: "{HOME}/cv/unified_pipeline/models/legibility/legibility_resnet34_hockey.pth"
  vitpose_checkpoint: "{HOME}/cv/unified_pipeline/models/vitpose/vitpose-h.pth"
  lock_threshold: 3.0

team:
  classifier_type: "hybrid"
  robust_model_name: "google/siglip-base-patch16-224"

multipass:
  enable_backward_pass: true
  enable_interpolation: true

device: "cuda"
verbose: true
"""

with open(f'{HOME}/cv/config.yaml', 'w') as f:
    f.write(config_yaml)

print(f"Configuration saved to {HOME}/cv/config.yaml")
print("\nConfig contents:")
print(config_yaml)

---
## Step 14: Run the Pipeline

In [ ]:
# Run tracking pipeline
OUTPUT_PATH = f"{HOME}/output_tracked.mp4"

!python -m unified_pipeline.cli \
    --video "{VIDEO_PATH}" \
    --output "{OUTPUT_PATH}" \
    --config {HOME}/cv/config.yaml

---
## Step 15: Download Results

In [ ]:
# Check outputs
!ls -lh {HOME}/output_tracked*

In [ ]:
# Download the tracked video
print("Downloading tracked video...")
files.download(f'{HOME}/output_tracked.mp4')

In [ ]:
# Download MOT format results (if generated)
mot_path = f'{HOME}/output_tracked.txt'
if os.path.exists(mot_path):
    print("Downloading MOT results...")
    files.download(mot_path)
else:
    print("No MOT file generated")

---
## Optional: Preview Video in Notebook

In [ ]:
# Preview first few frames
import cv2
import matplotlib.pyplot as plt

def show_frames(video_path, num_frames=4):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fig, axes = plt.subplots(1, num_frames, figsize=(20, 5))
    
    for i, frame_idx in enumerate([int(total * j / num_frames) for j in range(num_frames)]):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            axes[i].imshow(frame_rgb)
            axes[i].set_title(f'Frame {frame_idx}')
            axes[i].axis('off')
    
    cap.release()
    plt.tight_layout()
    plt.show()

print("Tracked video preview:")
show_frames(f'{HOME}/output_tracked.mp4')

---
## Alternative: Run Without Jersey/Team (Faster)

For quick detection + tracking only:

In [ ]:
# Fast mode: detection + tracking only
!python -m unified_pipeline.cli \
    --video "{VIDEO_PATH}" \
    --output {HOME}/output_fast.mp4 \
    --config {HOME}/cv/config.yaml \
    --no-jersey \
    --no-team

---
## Model Paths Quick Reference

| Model | Path |
|-------|------|
| YOLO Detection | `/content/cv/unified_pipeline/models/hockey_yolo.pt` |
| SAM2 Checkpoint | `/content/segment-anything-2-real-time/checkpoints/sam2.1_hiera_large.pt` |
| SAM2 Config | `/content/segment-anything-2-real-time/configs/sam2.1/sam2.1_hiera_l.yaml` |
| CUTIE | `/content/Cutie/weights/cutie-base-mega.pth` |
| ViTPose | `/content/cv/unified_pipeline/models/vitpose/vitpose-h.pth` |
| PARSeq | `/content/cv/unified_pipeline/models/parseq/parseq_hockey.ckpt` |
| Legibility | `/content/cv/unified_pipeline/models/legibility/legibility_resnet34_hockey.pth` |
| SigLIP | `google/siglip-base-patch16-224` (HuggingFace, auto-cached) |